In [1]:
import sys
!{sys.executable} -m pip install --user findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType, DoubleType

In [4]:
from scorers import score_text
from spark_tools import SparkSentimentStreamer

In [5]:
sc = SparkContext("local[*]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)
spark = SparkSession \
    .builder \
    .appName("SentimentWikiProcessor") \
    .getOrCreate()

In [6]:
dataInp = "streamInput/"
dataOut = "streamOut/"

In [7]:
streamer = SparkSentimentStreamer(sc, ssc, spark, score_text, dataInp, dataOut)

In [ ]:
streamer.run()

In [16]:
streamer.stop()

--------------------------------------------

# Example of score_text usage

In [8]:
from scorers import score_text

In [9]:
import sys
# legal dirty hack
sys.path.insert(0, '../ml/')
from LogisticRegressionCached import readFromCache

(lrModel, pipelineModel) = readFromCache('../ml/train/')

models loaded


In [11]:
query = "Amazon Company"

negative_paragraph = """{0} is very bad. And author doesn't provide any justification. People don't like {0}. 
Some even hate {0}, because {0} is evil. Some groups believe {0} is their main enemy.
""".format(query)

In [13]:
score_text(negative_paragraph, method='lr', pipelineModel=pipelineModel, lrModel=lrModel, spark=spark)

lr


-0.8152548728033254

In [15]:
score_text(negative_paragraph)

vader


-0.9186

In [16]:
score_text(negative_paragraph, method='lr-advanced', pipelineModel=pipelineModel, lrModel=lrModel, spark=spark)

lr-adv


-0.06745336021982078